In [1]:
import gc
import re
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn.preprocessing import StandardScaler
print(os.listdir("../input"))

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPooling1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM, CuDNNGRU, Bidirectional, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.constraints import maxnorm
from keras.callbacks import Callback
from keras import optimizers

Using TensorFlow backend.


In [3]:
def f1_score(true,pred): #considering sigmoid activation, threshold = 0.5
    pred = K.cast(K.greater(pred,0.5), K.floatx())

    groundPositives = K.sum(true) + K.epsilon()
    correctPositives = K.sum(true * pred) + K.epsilon()
    predictedPositives = K.sum(pred) + K.epsilon()

    precision = correctPositives / predictedPositives
    recall = correctPositives / groundPositives

    m = (2 * precision * recall) / (precision + recall)

    return m

In [4]:
train_df = pd.read_csv('../input/train.csv', usecols=['question_text', 'target'])
test_df = pd.read_csv('../input/test.csv', usecols = ['question_text'])

In [5]:
special_character_removal = re.compile(r'[^A-Za-z\.\-\?\!\,\#\@\% ]',re.IGNORECASE)

def clean_text(x):
    x_ascii = unidecode(x)
    x_clean = special_character_removal.sub('',x_ascii)
    return x_clean

In [6]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_text(str(x)))
test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_text(str(x)))

train_sentences = train_df['question_text']
train_labels = train_df['target']
test_sentences = test_df['question_text']

In [7]:
def add_features(df):
    
    df['question_text'] = df['question_text'].apply(lambda x:str(x))
    df['total_length'] = df['question_text'].apply(len)
    df['capitals'] = df['question_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/(float(row['total_length'])+1),
                                axis=1)
    df['num_words'] = df['question_text'].str.count('\S+')
    df['num_unique_words'] = df['question_text'].apply(lambda comment: len(set(w for w in comment.split())))
    df['words_vs_unique'] = df['num_unique_words'] / (df['num_words']+1)
    del df['num_unique_words'], df['num_words'], df['capitals'], df['total_length']
    return df

In [8]:
train_df = add_features(train_df)
test_df = add_features(test_df)

In [9]:
train_features = train_df[['caps_vs_length', 'words_vs_unique']].fillna(0)
test_features = test_df[['caps_vs_length', 'words_vs_unique']].fillna(0)

In [10]:
ss = StandardScaler()
ss.fit(np.vstack((train_features, test_features)))
train_features = ss.transform(train_features)
test_features = ss.transform(test_features)

In [11]:
gc.collect()

155

In [12]:
max_features = 20000
maxlen = 100

In [13]:
tokenizer = text.Tokenizer(num_words=max_features)

In [14]:
tokenizer.fit_on_texts(list(train_sentences) + list(test_sentences))

In [15]:
tokenized_train = tokenizer.texts_to_sequences(train_sentences)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [16]:
tokenized_test = tokenizer.texts_to_sequences(test_sentences)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [17]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'

In [18]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [19]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size)) #embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [20]:
del word_index, embeddings_index, all_embs, tokenized_test, tokenized_train, tokenizer, train_sentences, test_sentences, nb_words
gc.collect()

0

In [21]:
batch_size = 1024
epochs = 8
embed_size = 300

In [22]:
gc.collect()

0

In [23]:
def cudnnlstm_model(features):
    features_input = Input(shape=(features.shape[1],))
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(CuDNNLSTM(64, kernel_initializer='glorot_normal', return_sequences = True))(x)
    x, x_h, x_c = Bidirectional(CuDNNGRU(64, kernel_initializer='glorot_normal', return_sequences=True, return_state = True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, x_h, max_pool, features_input])
    x = Dense(32, activation="tanh", kernel_initializer='glorot_normal')(x)
    x = Dense(1, activation="sigmoid", kernel_initializer='glorot_normal')(x)
    model = Model(inputs=[inp,features_input], outputs=x)
    adam = optimizers.adam(clipvalue=1.0)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=[f1_score])

    return model

In [24]:
cudnnlstm_model = cudnnlstm_model(train_features)
cudnnlstm_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     6000000     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 128)     187392      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) [(None, 100, 128), ( 74496       bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_ave

In [25]:
weight_path="early_weights.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_f1_score', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor="val_f1_score", mode="max", patience=3)
callbacks = [checkpoint, early_stopping]

In [ ]:
cudnnlstm_model.fit([X_train, train_features], train_labels, batch_size=batch_size, epochs=2, shuffle = True, validation_split=0.20, callbacks=callbacks)

In [ ]:
cudnnlstm_model.load_weights(weight_path)
y_pred = cudnnlstm_model.predict([X_test,test_features], batch_size=batch_size)
y_pred = [x for i in y_pred for x in i]

In [ ]:
sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
sample['prediction'] = pd.Series(y_pred)

In [ ]:
sample['prediction'] = sample['prediction'].apply(lambda x: 0 if x <= 0.5 else 1)

In [ ]:
sample.to_csv('submission.csv', index=False)

In [ ]:
sample['prediction'].value_counts()